In [0]:
import torch #基本モジュール
from torch.autograd import Variable #自動微分用
import torch.nn as nn #ネットワーク構築用
import torch.optim as optim #最適化関数
import torch.nn.functional as F #ネットワーク用の様々な関数
import torch.utils.data #データセット読み込み関連
import torchvision #画像関連
from torchvision import datasets, models, transforms #画像用データセット諸々

import numpy as np
import argparse
import json
from logging.config import dictConfig
from logging import getLogger
import os
import time

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.AvgPool2d(8) #???
        )
        self.pc = nn.Sequential(
            nn.Linear(512, 1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 10),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 512)
        x = self.pc(x)
        return x

In [0]:
def train(model, device, train_loader, optimizer, criterion, logger):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            logger.debug("[train] batch : %s/%s (%.0f%%),\tloss : %.6f",
                batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item())
    return (None, loss.item())

In [0]:
def test(model, device, test_loader, criterion, logger):
    model.eval()
    test_loss = []
    correct = 0
    result = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss.append(criterion(output, target).item())
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            result += torch.eq(torch.max(output, 1).indices, target)
            
    test_loss = np.mean(np.array(test_loss))
    accuracy = 100. * correct / len(test_loader.dataset)
    
    logger.info("[test] ave loss : %.4f,\taccu : %d/%d(%.0f%%)",
        test_loss, correct, len(test_loader.dataset), accuracy)
    
    return (torch.tensor(result).numpy(), (test_loss, accuracy))

In [0]:
def main(description, option="", no_swap=False):
    with open('logging.json') as f:
        dictConfig(json.load(f))
    logger = getLogger('env')
    logger.debug("<" * 40)
    logger.info("[system] start")
    logger.info("[meta] %s", description)
    start_time = time.time()

    def fetch_args(args=[]):
        parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
        parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                            help='input batch size for training (default: 64)')
        parser.add_argument('--epochs', type=int, default=14, metavar='N',
                            help='number of epochs to train (default: 14)')
        parser.add_argument('--trials', type=int, default=10, metavar='T')
        parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
                            help='learning rate (default: 0.001)')
        parser.add_argument('--b1', type=float, default=0.9, metavar='B',
                            help='learning rate beta (default: 0.9)')
        parser.add_argument('--b2', type=float, default=0.999, metavar='B',
                            help='learning rate beta (default: 0.999)')
        parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                            help='learning rate momentum (default: 0.9)')
        parser.add_argument('--dataset', default="cifar10", metavar='D')
        parser.add_argument('--data-size', type=int, default=25000, metavar='D')
        parser.add_argument('--model', default="cnn", metavar='M')
        parser.add_argument('--optimizer', default="adam", metavar='O')
        parser.add_argument('--criterion', default="cross_entropy_loss", metavar='C')
        parser.add_argument('--scheduler', default="", metavar='S')
        parser.add_argument('--t-max', type=int, default=10, metavar='T')
        parser.add_argument('--save-model', action='store_true', default=False,
                            help='For Saving the current Model')
        return parser.parse_args(args=args)
  
    args = fetch_args(option.split(" "))
    for arg in vars(args):
        logger.info("[param] %s=%s", arg, vars(args)[arg])
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    logger.info("[device] %s", device)
    
    #画像の変形処理
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    #CIFAR-10のtrain, testsetのロード
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

    def subset_dataloader(set):
      return torch.utils.data.DataLoader(
          torch.utils.data.Subset(trainset, set), 
          batch_size=args.batch_size, **kwargs)

    # init
    N = int(args.data_size) #int(len(trainset) / 2)
    indices = np.arange(len(trainset))[:2 * N]
    np.random.shuffle(indices)
    A, B = indices[:N], indices[N:]

    labels = []
    for (_, target) in subset_dataloader(indices):
      labels += target.numpy().tolist()
    labels = np.array(labels)
    logger.debug("[debug] labels : %s", labels.tolist())
    
    log = {"X_train":[], "X_test":[], "Y_train":[], "Y_test":[], "X_acc":[], "Y_acc":[],
          "atob": [], "btoa": []}


    for trial in range(args.trials):
      logger.debug("-" * 20)
      logger.info("[system] trial %d / %d", trial, args.trials)

      # log
      log["X_train"].append([])
      log["X_test"].append([])
      log["X_acc"].append([])
      if not no_swap: 
        log["Y_train"].append([])
        log["Y_test"].append([])
        log["Y_acc"].append([])

      # instantiate
      if True:
        model_X = CNN().to(device)
        if not no_swap: model_Y = CNN().to(device)

        if args.optimizer == "adam":
          optimizer_X = optim.Adam(model_X.parameters(), lr=args.lr, betas=(args.b1, args.b2))
          if not no_swap: optimizer_Y = optim.Adam(model_Y.parameters(), lr=args.lr, betas=(args.b1, args.b2))
        if args.optimizer == "adagrad":
          optimizer_X = optim.Adagrad(model_X.parameters(), lr=args.lr)
          if not no_swap: optimizer_Y = optim.Adagrad(model_Y.parameters(), lr=args.lr)
        if args.optimizer == "sdg":
          optimizer_X = optim.SGD(model_X.parameters(), lr=args.lr, momentum=args.momentum)
          if not no_swap: optimizer_Y = optim.SGD(model_Y.parameters(), lr=args.lr, momentum=args.momentum)
        if args.optimizer == "asdg":
          optimizer_X = optim.ASGD(model_X.parameters(), lr=args.lr)
          if not no_swap: optimizer_Y = optim.ASGD(model_Y.parameters(), lr=args.lr)

        scheduler_X = None
        if args.scheduler == "cosine_annealing":
          scheduler_X = optim.lr_scheduler.CosineAnnealingLR(optimizer_X, T_max=args.t_max, eta_min=1e-6)

        criterion = nn.CrossEntropyLoss()

      # Split
      A_loader, B_loader = subset_dataloader(A), subset_dataloader(B)
      
      def fit_X():
        logger.info("[system] X : train_a -> test_b")
        for epoch in range(args.epochs):
          logger.debug("-" * 20)
          logger.info("[system] epoch %d", epoch)

          (_, loss_XA) = train(model_X, device, A_loader, optimizer_X, criterion, logger)
          (testB, (loss_XB, acc_X)) = test(model_X, device, B_loader, criterion, logger)
          # if scheduler_X: scheduler_X.step() # うごいてなさそう
        
          log["X_train"][trial].append(loss_XA)
          log["X_test"][trial].append(loss_XB)
          log["X_acc"][trial].append(acc_X)
        return testB

      def fit_Y():
        logger.info("[system] Y : train_b -> test_a")
        for epoch in range(args.epochs):
          logger.debug("-" * 20)
          logger.info("[system] epoch %d", epoch)

          (_, loss_YB) = train(model_Y, device, B_loader, optimizer_Y, criterion, logger)
          (testA, (loss_YA, acc_Y)) = test(model_Y, device, A_loader, criterion, logger)
        
          log["Y_train"][trial].append(loss_YB)
          log["Y_test"][trial].append(loss_YA)
          log["Y_acc"][trial].append(acc_Y)
        return testA

      # training
      testB = fit_X()
      if not no_swap: testA = fit_Y()

      # swap
      if not no_swap:
        # result
        A_o, A_x = A[testA], A[np.logical_not(testA)]
        B_o, B_x = B[testB], B[np.logical_not(testB)]

        # label
        A_label = [(i, labels[i]) for i in A_x]
        B_label = [(i, labels[i]) for i in B_o]
        logger.debug("[debug] A_x labels %s", A_label)
        logger.debug("[debug] B_o labels %s", B_label)
        
        # class batch
        A_trans, A_left = [], []
        B_trans, B_left = [], []
        for class_idx in range(10):
          A_class = [i for (i, label) in A_label if label==class_idx]
          B_class = [i for (i, label) in B_label if label==class_idx]
          s = min(len(A_class), len(B_class))
          A_trans += B_class[:s]
          A_left += A_class[s:]
          B_trans += A_class[:s]
          B_left += B_class[s:]
        logger.debug("[debug] B to A %s", A_trans)
        logger.debug("[debug] A to B %s", B_trans)
        
        # concat
        A = np.concatenate([A_o, A_trans, A_left]).astype(np.int64)
        B = np.concatenate([B_x, B_trans, B_left]).astype(np.int64)
        np.random.shuffle(A)
        np.random.shuffle(B)
        
      # log
      if not no_swap: 
        log["atob"].append(np.copy(A_trans).tolist())
        log["btoa"].append(np.copy(B_trans).tolist())

    # Statistics
    logger.debug("%s statistics %s", "-" * 10, "-" * 10)
    logger.info("[stat] X train loss : %s", log["X_train"])
    logger.info("[stat] X test loss : %s", log["X_test"])
    logger.info("[stat] X accuracy : %s", log["X_acc"])
    if not no_swap:
      logger.info("[stat] Y train loss : %s", log["Y_train"])
      logger.info("[stat] Y test loss : %s", log["Y_test"])
      logger.info("[stat] Y accuracy : %s", log["Y_acc"])
      logger.info("[stat] swap log A to B : %s", log["atob"])
      logger.info("[stat] swap log B to A : %s", log["btoa"])
    logger.info("[stat] elapsed time : %s[s]", time.time() - start_time)
    
    if args.save_model:
        logger.info("[system] saving...")
        torch.save(model.state_dict(), "swap.pt")
    
    logger.info("[system] finish")


In [0]:
def download_log():
  from google.colab import files
  files.download("./log.txt")

In [0]:
if __name__ == '__main__':
    main("デバッグ",
         option="--epochs 3 --batch-size 16 --optimizer sdg --data-size 100 --trials 2",
         no_swap=False)

In [15]:
if __name__ == '__main__':
    main("データスワップ : main",
         option="--epochs 30 --batch-size 16 --optimizer sdg --data-size 5000 --trials 10")

[system] start
[meta] データスワップ : main
[param] batch_size=16
[param] epochs=30
[param] trials=10
[param] lr=0.001
[param] b1=0.9
[param] b2=0.999
[param] momentum=0.9
[param] dataset=cifar10
[param] data_size=5000
[param] model=cnn
[param] optimizer=sdg
[param] criterion=cross_entropy_loss
[param] scheduler=
[param] t_max=10
[param] save_model=False
[device] cuda


Files already downloaded and verified


[system] trial 0 / 10
[system] X : train_a -> test_b
[system] epoch 0
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
[test] ave loss : 2.3026,	accu : 566/5000(11%)
[system] epoch 1
[test] ave loss : 2.3024,	accu : 485/5000(10%)
[system] epoch 2
[test] ave loss : 2.3019,	accu : 510/5000(10%)
[system] epoch 3
[test] ave loss : 2.2952,	accu : 519/5000(10%)
[system] epoch 4
[test] ave loss : 2.2820,	accu : 838/5000(17%)
[system] epoch 5
[test] ave loss : 2.2723,	accu : 903/5000(18%)
[system] epoch 6
[test] ave loss : 2.2543,	accu : 904/5000(18%)
[system] epoch 7
[test] ave loss : 2.2430,	accu : 914/5000(18%)
[system] epoch 8
[test] ave loss : 2.2687,	accu : 776/5000(16%)
[system] epoch 9
[test] ave loss : 2.2850,	accu : 682/5000(14%)
[system] epoch 10
[test] ave loss : 2.2946,	accu : 633/5000(13%)
[system] epoch 11
[test

In [0]:
if __name__ == '__main__':
    main("データを分割して交換しないアルゴリズム / ベースライン / deep cnn",
        #  option="--epochs 15 --batch-size 16 --optimizer sdg --data-size 25000", # 65%
        #  option="--epochs 100 --batch-size 16 --optimizer sdg --data-size 5000", # 65%
         option="--epochs 200 --batch-size 16 --optimizer sdg --data-size 5000", # %
         
        #  option="--epochs 15 --batch-size 16 --optimizer adagrad --lr 0.001 --data-size 25000", # 57%
        #  option="--epochs 15 --batch-size 16 --optimizer asdg --lr 0.00005 --data-size 25000", # 10%
         
        #  option="--epochs 100 --batch-size 32 --optimizer adam --scheduler=cosine-annealing --lr=0.00005 --b1=0.85 --b2=0.99 --data-size 5000", # 57%
        #  option="--epochs 100 --batch-size 32 --optimizer adam --scheduler= --lr=0.00005 --b1=0.85 --b2=0.99 --data-size 5000", # 55%

        #  option="--epochs 100 --batch-size 32 --optimizer adam --lr=0.00005 --b1=0.85 --b2=0.99 --data-size 5000", # 55%
        #  option="--epochs 25 --batch-size 32 --optimizer adam --lr=0.00005 --b1=0.8 --b2=0.9 --data-size 25000",
        #  option="--epochs 100 --batch-size 32 --optimizer adam --lr=0.00003 --b1=0.8 --b2=0.9 --data-size 5000", # 55%
        #  option="--epochs 25 --batch-size 16 --optimizer adam --lr=0.00005 --b1=0.8 --b2=0.9 --data-size 5000",
         
        #  option="--epochs 25 --batch-size 64 --optimizer adam --lr=0.0001 --b1=0.6 --b2=0.8 --data-size 5000",
        #  option="--epochs 10 --batch-size 64 --optimizer adam --lr=0.0001 --b1=0.5 --b2=0.7 --data-size 5000",
         no_swap=True)
    
    download_log()

[system] start
[meta] データを分割して交換しないアルゴリズム / ベースライン / deep cnn
[param] batch_size=16
[param] epochs=200
[param] lr=0.001
[param] b1=0.9
[param] b2=0.999
[param] momentum=0.9
[param] dataset=cifar10
[param] data_size=5000.0
[param] model=cnn
[param] optimizer=sdg
[param] criterion=cross_entropy_loss
[param] scheduler=
[param] save_model=False
[device] cuda


Files already downloaded and verified


[system] epoch 0
[system] train_a -> test_b
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
[test] ave loss : 2.3025,	accu : 497/5000(10%)
[system] epoch 1
[system] train_a -> test_b
[test] ave loss : 2.3024,	accu : 504/5000(10%)
[system] epoch 2
[system] train_a -> test_b
[test] ave loss : 2.3023,	accu : 530/5000(11%)
[system] epoch 3
[system] train_a -> test_b
[test] ave loss : 2.3020,	accu : 625/5000(12%)
[system] epoch 4
[system] train_a -> test_b
[test] ave loss : 2.3011,	accu : 787/5000(16%)
[system] epoch 5
[system] train_a -> test_b
[test] ave loss : 2.2948,	accu : 824/5000(16%)
[system] epoch 6
[system] train_a -> test_b
[test] ave loss : 2.2753,	accu : 862/5000(17%)
[system] epoch 7
[system] train_a -> test_b
[test] ave loss : 2.2589,	accu : 856/5000(17%)
[system] epoch 8
[system] train_a -> test_b
[test] av

In [0]:
if __name__ == '__main__':
    main("データを分割して交換するアルゴリズム / main / deep cnn",
         option="--epochs 50 --batch-size 16 --optimizer sdg")

[system] start
[meta] データを分割して交換するアルゴリズム / main / deep cnn
[param] batch_size=16
[param] epochs=50
[param] lr=0.001
[param] b1=0.9
[param] b2=0.999
[param] momentum=0.9
[param] dataset=cifar10
[param] model=cnn
[param] optimizer=sdg
[param] criterion=cross_entropy_loss
[param] save_model=False
[device] cuda


Files already downloaded and verified


[system] epoch 0
[system] train_a -> test_b
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


s swa


[system] train_b -> test_a
[system] epoch 1
[system] train_a -> test_b


...

swapping
s swa


[system] train_b -> test_a
[system] epoch 49
[system] train_a -> test_b


swapping
s swa


[system] train_b -> test_a
[stat] X train loss : [2.222242832183838, 2.196442127227783, 2.2245125770568848, 2.2336411476135254, 2.1939828395843506, 2.204033851623535, 2.187859296798706, 2.1586098670959473, 2.1380679607391357, 2.0317063331604004, 2.4475395679473877, 2.3039584159851074, 1.897559404373169, 1.8836838006973267, 1.9662189483642578, 2.1247215270996094, 1.8901077508926392, 1.7523423433303833, 1.8175065517425537, 1.7399953603744507, 2.3118703365325928, 2.0122146606445312, 1.9611204862594604, 1.8403459787368774, 1.8354214429855347, 1.8361092805862427, 2.4611496925354004, 2.3354556560516357, 1.6978340148925781, 1.63152015209198, 1.7064489126205444, 1.677284836769104, 1.8245878219604492, 1.7409210205078125, 1.798946499824524, 2.203552007675171, 1.7661759853363037, 1.8074859380722046, 1.9536830186843872, 1.9538627862930298, 1.952601432800293, 1.950279951095581, 1.9495999813079834, 1.9476701021194458, 1.9467227458953857, 1.9474114179611206, 2.0966269969940186, 2.2114222049713135, 1.

swapping


In [0]:
if __name__ == '__main__':
    main("データを分割して交換するアルゴリズム / no_train / deep cnn",
         option="--epochs 25 --batch-size 256",
         no_train=True)